In [ ]:
'''해야할 일
   1)검정의 가정 조사하기
   2)Welch T-test 조사하기
'''

## Chapter 6. Student's t-test
수치를 비교할 때 단순히 차이만 볼 경우 주관이 개입되는 문제가 있다. 예를 들어, 측정단위를 g으로 할 때 수치간 차이가 6kg이 나올 경우 차이가 작다 느낄 수 있지만 g으로 측정할 경우 6000g으로 차이가 크다 판단할 수 있다. 또 누군가는 6kg의 차이가 작다 느끼고 누군가는 크다 느껴 사람간 해석의 차이가 존재한다.<br>
문제점들을 보완하여 통계적으로 차이가 유의한지 검정하는 방법이 t-test이다. t-test는 집단(범주)의 수치형 변수 `평균`을 비교하는 분석 방법으로 아래와 같이 구분한다.

In [1]:
import pandas as pd
data = pd.read_csv('data/Ashopping.csv', encoding='cp949')

In [ ]:
# t-test는 표본 개수가 30개 미만일 경우 정규성 검정을 수행해야 한다.
# t-test는 비율의 차이도 분석하는가?

### To be Satisfied
- 검정하려는 변수는 연속형 변수이며 각 개체는 확률표본이어야 한다.
- 정규성을 만족해야 한다.<br>
  ※ 표본수가 적어 정규성 검정이 어렵다면 비모수 검정(Wilcoxon 부호 순위 검정)을 시도해야 한다.

In [ ]:
# p 값은 기본 모집단 평균이 실제로 20일 때 표본 평균이 21.4에 이르는 극단값을 보일 가능도, 
# 다시 말해서 표본에서 관측한 평균과 다르거나 혹은 모집단 평균 20과 훨씬 다른 표본 평균이 관측될 확률을 설명합니다. 
# p 값이 0.0046이면 10,000을 기준으로 가능한 횟수가 대략 46임을 의미합니다. 
# 따라서 모집단 평균이 20과 같다는 귀무가설을 의심없이 기각할 수 있습니다.

### $\blacktriangleright$ One-Sample T-test
$~~$ 알려진 모집단의 평균($\mu_0$)과 표본평균($\bar{X}$)이 통계적으로 일치하다 볼 수 있는 지 확인하는 기법으로 2022년 예상 판매액과 실제 판매액 간에 차이가 있는지,<br>
$~~$ A회사 직원의 평균 키가 172cm로 알려졌는데 실제 직원들의 평균 키의 평균이 같은지와 같이 예상과 사실이 유효한지 파악할 때 사용한다.<br>

- 검정통계량 $t$는 평균 추정값과 실제 평균값의 차이가 표준오차에 비해 몇 배 더 큰지를 의미해 $t$가 클수록 $\mu_0$와 $\overline{X}$의 차이가 더 유효하다.
$$\large t_{n-1} = \frac{\overline{X} - \mu_0}{S / \sqrt{\Large n}} \\ \small\mu_0\text{: 이론적 평균, }\overline{X}\text{: 표본의 평균, }\text{S: 표본의 표준편차, n: 표본의 수}$$
<br>
- p-value와 유의수준 $\alpha$에 의해 기각$\cdot$채택되는 귀무가설, 대립가설은 아래와 같다.
$$H_0: \overline{X}=\mu_0, H_1: \overline{X}\neq\mu_0$$
- 일반적으로 위와 같은 양측검정을 시도하나 검정하고자 하는 가설이 평균 추정값보다 실제 평균값이 크다, 작다라는 것이라면 단측검정을 수행한다.

$~~$ ※ 한 집단의 평균을 특정값과 비교하는 방법에는 Z검정도 있으나 Z검정은 모분산이 알려졌고 표본의 수가 많을 때($n > 30$) 사용하는 방법으로 <br>
$~~~~~$ 실제 분석상황에선 사용하기 어렵다.

```python
ttest_1samp(X, MU, alternative, nan_policy)
o alternative = 'less'           단측검정(H1: X.mean() < MU)을 수행한다.
  alternative = 'greater'        단측검정(H1: X.mean() > MU)을 수행한다.
  alternative = 'two-sided'      양측검정(H1: X.mean() != MU)을 수행한다.
o nan_policy = 'propagate'(returns nan)|'omit'(ignoring nan)
```

In [6]:
# 총_매출액의 평균이 7700000과 같은지 검정했을 때 0.01하에서 귀무가설을 기각한다. 즉, 총_매출액은 7700000과 다르다.
from scipy.stats import ttest_1samp

ttest_1samp(data['총_매출액'], 7700000)

Ttest_1sampResult(statistic=-2.8657430145836007, pvalue=0.9978762069258367)

### $\blacktriangleright$ Two-Sample T-test
독립된 두 집단의 수치형 변수 평균이 통계적으로 일치하다 볼 수 있는 지 확인하는 기법으로 남녀별 구매액 차이, 유지고객과 이탈고객간 구매빈도 차이를 평균으로 비교하는 방법이다.
- 두 집단의 분산이 같음을 가정해 등분산성 검정을 t-test에 앞서 수행한다.
  - Bartlett: 데이터가 정규분포일 때 사용하는 등분산성 검정으로 F검정통량을 사용한다.
  - levene: 데이터가 정규분포를 따르지 않을 때 사용하는 등분산성 검정
  - fligner: 데이터의 정규성을 논하기 어려울 때 사용하는 비모수 등분산성 검정
  
  $$\large H_0: \sigma_1^2 = \sigma_2^2, ~H_1: \sigma_1^2 \neq \sigma_2^2$$
  <br>
- 검정통계량 $t$는 두 표본의 평균 차이값($\overline{X}-\overline{Y}$)간 편차가 표준오차에 비해 몇 배 더 큰지 나타낸다.

$$\large t_{n_1+n_2-2} = \frac{\overline{X_1} - \overline{X_2}}{S\normalsize\sqrt{\frac{\large 1}{\Large n_1}+\frac{\large 1}{\Large n_2}}}$$$$(\small \overline{X_i}: X_i\text{집단의 표본평균}, ~S\text{: 두 집단의 표준편차}, ~n_i : X_i\text{집단의 표본수})$$
<br>
- p-value와 유의수준 $\alpha$에 의해 기각$\cdot$채택되는 귀무가설, 대립가설은 아래와 같다.
$$H_0: \overline{X_1}=\overline{X_2}, H_1: \overline{X_1}\neq\overline{X_2}$$
- 일반적으로 수행되는 가설검정은 위와 같으나 한 집단의 평균이 다른 집단의 평균보다 크다, 작다를 입증하고 싶을 경우 단측검정을 수행한다.

### $\blacktriangleright$ Welch's T-test
Two-Sample T-test에서 등분산 조건이 만족하지 않을 때 수행하는 평균 비교 분석법이다.
- 검정통계량 $t$
 $$\large t_{df} = \frac{\overline{X_1} - \overline{X_2}}{\normalsize\sqrt{\frac{\large S_1^2}{\Large n_{\small1}}+\frac{\large S_2^2}{\Large n_{\small2}}}}, ~ _{df} = \frac{\left(\frac{S_1^2}{n_1}+\frac{S_2^2}{n_2}\right)^2}{\frac{(S_1^2/n_1)^2}{n_1 - 1}+\frac{(S_2^2/n_2)^2}{n_2 - 1}}$$

In [7]:
# 1) 등분산 검정
#  :유의수준 0.01 하에서 귀무가설이 기각된다. 두 표본은 등분산을 만족하지 않는다.
from scipy.stats import bartlett
bartlett(data.query('클레임접수여부==0').방문빈도, data.query('클레임접수여부==1').방문빈도)

BartlettResult(statistic=13.626177910964618, pvalue=0.0002230534980645927)

In [8]:
# 2) 독립표본 t-test
#  :유의수준 0.01 하에서 귀무가설을 기각한다. 즉, 두 표본의 평균은 다르다.
from scipy.stats import ttest_ind
ttest_ind(data.query('클레임접수여부==0').방문빈도, data.query('클레임접수여부==1').방문빈도, equal_var=False)

Ttest_indResult(statistic=2.595726838875684, pvalue=0.009577734932789503)

### $\blacktriangleright$ Paired Sample T-test
한 집단의 실험 전후 결과간, 서로 다른 상황에서의 평균이 통계적으로 일치하다 볼 수 있는 지 확인하는 기법이다. 가령 시험 난이도가 똑같을 때 중간고사와 기말고사의 시험평균이 일치하는 지 비교하는 것이다.
- 실험을 통해 얻어낸 표본들로 시점이 상이해 독립성, 등분산 검정 모두 필요하지 않다.
- 검정통계량 $t$는 두 쌍체표본 실험 전후 차이가 표준오차에 비해 몇 배 더 큰 지를 의미한다.

  $$D=X_1 - X_2, ~~\large t = \frac{\overline{D}}{S_D ~/ \sqrt{\Large n}}$$
$$(\overline{D}: D\text{의 표본평균}, ~ S_D: D\text{의 표본표준편차})$$
- 개체당 두 개의 데이터가 있어야 하나 하나만 존재하는, 결측이 있는 경우에 삭제, 표본평균으로 대치, 값을 추청해 대치, 여러 대치 후보값 중 무작위로 선택해 대치, 검증하려는 변수 이외의 변수들이 동일한 데이터의 값들 중 무작위로 선택해 대치하는 방법(Hot-deck)이 있다.
- 일반적인 가설은 아래와 같으나 때에 따라 세운 가설이 두 표본의 차이 평균이 특정값보다 크다, 작다이면 단측검정을 하기도 한다.
  $$H_0: \mu_D = 0, H_1: \mu_D \neq 0$$

In [ ]:
# 1) D의 정규성 검정
# :데이터의 수가 30 이상으로 만족한다 가정한다.

In [9]:
# 2) Paired-ttest
# :유의수준 0.01 하에서 귀무가설을 기각한다.
#  즉 멤버쉽 프로그램 가입으로 인해 만족도 차이가 존재하며 t값에 따라 만족도가 더 증가한 것으로 보여진다.
from scipy.stats import ttest_rel
ttest_rel(data['멤버쉽_프로그램_가입전_만족도'], data['멤버쉽_프로그램_가입후_만족도'])

Ttest_relResult(statistic=-29.560410783358122, pvalue=1.7319140513197275e-138)

### $\blacktriangleright$ Mann-Whitney U-test

In [ ]:
# 집단이 3개 이상일 땐 ANOVA를 사용한다.